# Birthdays probing test - pre-trained TAPAS

In [1]:
import sys
sys.path.append('/home/jxm3/research/deidentification/unsupervised-deidentification')

In [2]:
from model import CoordinateAscentModel

import os

num_cpus = len(os.sched_getaffinity(0))

model = CoordinateAscentModel(
    document_model_name_or_path='roberta-base',
    profile_model_name_or_path='google/tapas-base',
    num_workers=min(8, num_cpus),
    learning_rate=1e-5,
    pretrained_profile_encoder=False,
    word_dropout_ratio=0.0,
    word_dropout_perc=0.0,
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialized model with learning_rate = 1e-05 and patience 3


In [ ]:
from dataloader import WikipediaDataModule
import os

num_cpus = os.cpu_count()

dm = WikipediaDataModule(
    document_model_name_or_path='roberta-base',
    profile_model_name_or_path='google/tapas-base',
    max_seq_length=128,
    dataset_name='wiki_bio',
    dataset_train_split='train[:100%]',
    dataset_val_split='val[:100%]',
    dataset_test_split='test[:100%]',
    dataset_version='1.2.0',
    num_workers=num_cpus,
)
dm.setup("fit")

Initializing WikipediaDataModule with num_workers = 32 and mask token `<mask>`
loading wiki_bio[1.2.0] split train[:100%]


Using custom data configuration default
Reusing dataset wiki_bio (/home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da)


loading wiki_bio[1.2.0] split val[:100%]


Using custom data configuration default
Reusing dataset wiki_bio (/home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da)


loading wiki_bio[1.2.0] split test[:100%]


...

#27:   0%|          | 0/18208 [00:00<?, ?ex/s]

#28:   0%|          | 0/18208 [00:00<?, ?ex/s]

#29:   0%|          | 0/18208 [00:00<?, ?ex/s]

#30:   0%|          | 0/18208 [00:00<?, ?ex/s]

#31:   0%|          | 0/18208 [00:00<?, ?ex/s]

## Get the birthday data

In [6]:
from typing import Any, Dict, List, Tuple

import datasets

import datetime
import re


def process_dataset_example(ex: Dict) -> Dict[str, Any]:
    """Parses example. Returns tuple (idx, month, day) or None."""
    profile = ex['profile']
    date_str_matches = re.search(r"birth_date \|\| ([\d]{1,4} [a-z]+ [\d]{1,4})", profile)
    try:
        date_str = date_str_matches.group(1)
        dt = datetime.datetime.strptime(date_str, "%d %B %Y")
        ex['birthday'] = (ex['text_key_id'], dt.month-1, dt.day-1)
    except (AttributeError, ValueError) as e:
        # no birthdate or date in unknown format (just skip these ones)
        ex['birthday'] = None
    return ex

def process_dataset(dataset: datasets.Dataset) -> List[Tuple[int, int, int]]:
    new_dataset = dataset.map(process_dataset_example)
    filtered_dataset = new_dataset.filter(lambda ex: ex['birthday'])
    return filtered_dataset['birthday']


In [12]:
import numpy as np
import torch
from tqdm.notebook import tqdm

def forward_profile(model: CoordinateAscentModel, batch: Dict[str, torch.Tensor], profile_key: str = 'profile', collapse_axis: bool = False) -> torch.Tensor:
    """Tokenizes text and inputs to profile encoder."""
    if torch.cuda.is_available(): assert model.profile_model_device.type == 'cuda'
    inputs = model._get_inputs_from_prefix(batch=batch, prefix=profile_key)
    if collapse_axis: # Collapse shape (b, n, s) -> (b * n, s)
        inputs = {
            k: v.flatten(end_dim=1)
            for k,v in inputs.items()
        }
    inputs = {k: v.to(model.profile_model_device) for k,v in inputs.items()}
    profile_outputs = model.profile_model(**inputs)
    profile_embeddings = profile_outputs['last_hidden_state']
    # mean-pooling across profile embeddings
    return profile_embeddings.mean(dim=1)

def precompute_profile_embeddings(model: CoordinateAscentModel, dm: WikipediaDataModule):
    model.profile_model.cuda()
    model.profile_model.eval()
    model.profile_embed.cuda()
    model.profile_embed.eval()

    profile_dim = 768

    train_profile_embeddings = np.zeros((len(dm.train_dataset), profile_dim))
    for train_batch in tqdm(dm.train_dataloader(), desc="Precomputing train embeddings", colour="#008080", leave=False):
        with torch.no_grad():
            profile_embeddings = forward_profile(model, batch=train_batch)
        train_profile_embeddings[train_batch["text_key_id"]] = profile_embeddings.cpu()
    train_profile_embeddings = torch.tensor(train_profile_embeddings, dtype=torch.float32)
    
    val_profile_embeddings = np.zeros((len(dm.val_dataset), profile_dim))
    for val_batch in tqdm(dm.val_dataloader()[0], desc="Precomputing val embeddings", colour="#c09591", leave=False):
        with torch.no_grad():
            profile_embeddings = forward_profile(model, batch=val_batch)
        val_profile_embeddings[val_batch["text_key_id"]] = profile_embeddings.cpu()
    val_profile_embeddings = torch.tensor(val_profile_embeddings, dtype=torch.float32)
    
    test_profile_embeddings = np.zeros((len(dm.val_dataset), profile_dim))
    for test_batch in tqdm(dm.test_dataloader(), desc="Precomputing test embeddings", colour="yellow", leave=False):
        with torch.no_grad():
            profile_embeddings = forward_profile(model, batch=test_batch)
        test_profile_embeddings[test_batch["text_key_id"]] = profile_embeddings.cpu()
    test_profile_embeddings = torch.tensor(test_profile_embeddings, dtype=torch.float32)
    
    return {
        'train': train_profile_embeddings,
        'val': val_profile_embeddings,
        'test': test_profile_embeddings,
    }



precompute_profile_embeddings(model, dm)

Precomputing train embeddings:   0%|          | 0/18209 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fb60b8570d0>
Traceback (most recent call last):
  File "/home/jxm3/.conda/envs/torch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    Exception ignored in: self._shutdown_workers()<function _MultiProcessingDataLoaderIter.__del__ at 0x7fb60b8570d0>

Traceback (most recent call last):
  File "/home/jxm3/.conda/envs/torch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
      File "/home/jxm3/.conda/envs/torch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
if w.is_alive():
      File "/home/jxm3/.conda/envs/torch/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
self._shutdown_workers()
  File "/home/jxm3/.conda/envs/torch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
        assert self._parent_pid == os.getpid(), 'can only test a chil

Precomputing test embeddings:   0%|          | 0/2276 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fb60b8570d0>
Traceback (most recent call last):
  File "/home/jxm3/.conda/envs/torch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    Exception ignored in: Exception ignored in: self._shutdown_workers()<function _MultiProcessingDataLoaderIter.__del__ at 0x7fb60b8570d0>

  File "/home/jxm3/.conda/envs/torch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fb60b8570d0>Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fb60b8570d0>  File "/home/jxm3/.conda/envs/torch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__

Exception ignored in: 
    Exception ignored in:     <function _MultiProcessingDataLoaderIter.__del__ at 0x7fb60b8570d0>Traceback (most recent call last):

if w.is_alive():

In [ ]:
from typing import Dict

import torch
import torchmetrics
import transformers

from pytorch_lightning import LightningModule
from transformers import AdamW

class BirthdayModel(LightningModule):
    """Probes the PROFILE for birthday info."""
    profile_embeddings: torch.Tensor
    classifier: torch.nn.Module
    learning_rate: float
    
    def __init__(self, model: CoordinateAscentModel, learning_rate: float):
        super().__init__()
        # We can pre-calculate these embeddings bc
        self.train_profile_embeddings = torch.tensor(model.train_profile_embeddings.cpu())
        self.val_profile_embeddings = torch.tensor(model.val_profile_embeddings.cpu())
        self.month_classifier = torch.nn.Sequential(
            torch.nn.Linear(model.bottleneck_embedding_dim, 64),
            # torch.nn.Dropout(p=0.01),
            torch.nn.Linear(64, 12),
        )
        self.day_classifier = torch.nn.Sequential(
            torch.nn.Linear(model.bottleneck_embedding_dim, 64),
            # torch.nn.Dropout(p=0.01),
            torch.nn.Linear(64, 31),
        )
        self.learning_rate = learning_rate
        self.train_accuracy = torchmetrics.Accuracy()
        self.val_accuracy   = torchmetrics.Accuracy()
        self.loss_criterion = torch.nn.CrossEntropyLoss()

    def training_step(self, batch: Tuple[int, int], batch_idx: int) -> torch.Tensor:
        profile_idxs, months, days = batch
        assert ((0 <= profile_idxs) & (profile_idxs < len(self.train_profile_embeddings))).all()
        assert ((0 <= months) & (months < 12)).all()
        assert ((0 <= days) & (days < 31)).all()
        
        clf_device = next(self.month_classifier.parameters()).device
        with torch.no_grad():
            embedding = self.train_profile_embeddings[profile_idxs].to(clf_device)
        
        
        month_logits = self.month_classifier(embedding)
        day_logits = self.day_classifier(embedding)
        
        
        month_loss = torch.nn.functional.cross_entropy(month_logits, months)
        day_loss = torch.nn.functional.cross_entropy(day_logits, days)
        
        self.log('train_acc_month', self.train_accuracy(month_logits, months))
        self.log('train_acc_day', self.train_accuracy(day_logits, days))
        
        return (month_loss + day_loss)
    
    def validation_step(self, batch: Dict[str, torch.Tensor], batch_idx: int):
        profile_idxs, months, days = batch
        assert ((0 <= profile_idxs) & (profile_idxs < len(self.val_profile_embeddings))).all()
        assert ((0 <= months) & (months < 12)).all()
        assert ((0 <= days) & (days < 31)).all()
        
        clf_device = next(self.month_classifier.parameters()).device
        with torch.no_grad():
            embedding = self.val_profile_embeddings[profile_idxs].to(clf_device)
        
        
        month_logits = self.month_classifier(embedding)
        day_logits = self.day_classifier(embedding)
        
        
        month_loss = torch.nn.functional.cross_entropy(month_logits, months)
        day_loss = torch.nn.functional.cross_entropy(day_logits, days)
        
        self.log('val_acc_month', self.val_accuracy(month_logits, months))
        self.log('val_acc_day', self.val_accuracy(day_logits, days))
        
        if batch_idx == 0:
            print('val_acc_month', self.val_accuracy(month_logits, months).item(), '//', 'val_acc_day', self.val_accuracy(day_logits, days).item())

        return (month_loss + day_loss)

    def configure_optimizers(self):
        """Prepare optimizer and schedule (linear warmup and decay)"""
        optimizer = AdamW(
            list(self.parameters()), lr=self.learning_rate
        )
        return optimizer
            

## Train it

In [ ]:
from pytorch_lightning import Trainer, seed_everything

seed_everything(42)

num_validations_per_epoch = 4

In [ ]:
birthday_model = BirthdayModel(model, 1e-4)
birthday_dm.batch_size = 2048

# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
trainer = Trainer(
    default_root_dir=f"saves/jup/birthday_probing",
    val_check_interval=1.0,
    max_epochs=100,
    log_every_n_steps=50,
    gpus=torch.cuda.device_count(),
)

In [ ]:
trainer.fit(birthday_model, birthday_dm)